<a href="https://colab.research.google.com/github/0x416C657852/DiceClassificationCNN/blob/main/DiceClassificationCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test